In [1]:
# Cell 1: Install dependencies
!pip install fastai duckduckgo_search --quiet

In [3]:
# Cell 2: Imports
from fastai.vision.all import *
from duckduckgo_search import DDGS
from fastcore.all import *

In [4]:
def search_images(keywords, max_images=200):
    print(f"Searching for '{keywords}'")
    with DDGS() as ddgs:
        return L([item['image'] for item in ddgs.images(keywords, max_results=max_images)])

In [5]:
# Cell 4: Download images
bear_types = 'grizzly bear','black bear','teddy bear'
path = Path('bears')

for o in bear_types:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'{o} photo'))
    resize_images(path/o, max_size=400, dest=path/o)


Searching for 'grizzly bear photo'
Searching for 'black bear photo'
Searching for 'teddy bear photo'


In [6]:
# Cell 5: Remove bad images
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
print(f"Deleted {len(failed)} corrupt images")

Deleted 39 corrupt images


In [7]:
# Cell 6: Create DataLoaders
bears = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

dls = bears.dataloaders(path)

In [8]:
# Cell 7: Create and train model
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\spenc/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 99.5MB/s]


epoch,train_loss,valid_loss,error_rate,time
0,1.492220,0.452439,0.107843,00:01


epoch,train_loss,valid_loss,error_rate,time
0,0.449508,0.365626,0.078431,00:00
1,0.354412,0.341424,0.068627,00:00
2,0.267693,0.336622,0.068627,00:00
3,0.218697,0.318562,0.068627,00:00


In [9]:
# Cell 8: Export model
learn.export('model.pkl')